In [1]:
import numpy as np
from scipy import interpolate, stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests import get
import json
import matplotlib.pyplot as plt
from collections import Counter
import re
import os
from dateutil.parser import parse
import operator
from uuid import uuid4
import sys
import geopandas
import shapefile

In [95]:
#load in data

ld = pd.read_csv('data/joined_drage_16_3.csv')


/Users/d/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (42,46,50,51,57,93,103) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [96]:
#check out which columns have values, drop the columns with no values
columns = ld.columns.tolist()

i = 0
for c in columns:
    
    nonan = len(ld[c].dropna().tolist())
    print(i, c,':',nonan)
    i+=1
    
    if nonan == 0:
        #print('Dropped:', c)
        ld.drop(c, inplace=True, axis=1)
        
  

0 geoid : 384692
1 sourceagen : 0
2 parcelnumb : 384560
3 usecode : 363097
4 usedesc : 0
5 zoning : 0
6 zoning_des : 0
7 struct : 0
8 multistruc : 0
9 structno : 0
10 yearbuilt : 0
11 numstories : 0
12 numunits : 0
13 structstyl : 0
14 parvaltype : 0
15 improvval : 363080
16 landval : 363080
17 parval : 363080
18 agval : 0
19 saleprice : 0
20 saledate : 0
21 taxamt : 0
22 owntype : 0
23 owner : 369485
24 ownfrst : 0
25 ownlast : 0
26 owner2 : 74581
27 owner3 : 0
28 owner4 : 0
29 subsurfown : 0
30 subowntype : 0
31 mailadd : 363112
32 mail_addre : 0
33 careof : 0
34 mail_addno : 0
35 mail_addpr : 0
36 mail_addst : 0
37 mail_add_1 : 0
38 mail_add_2 : 0
39 mail_unit : 0
40 mail_city : 355923
41 mail_state : 357165
42 mail_zip : 357168
43 mail_urban : 0
44 address : 359309
45 address2 : 0
46 saddno : 359309
47 saddpref : 282571
48 saddstr : 359309
49 saddsttyp : 358009
50 saddstsuf : 1387
51 sunit : 6604
52 scity : 359309
53 original_a : 359217
54 city : 384692
55 county : 384692
56 state2

In [97]:
ld.columns.tolist()

['geoid',
 'parcelnumb',
 'usecode',
 'improvval',
 'landval',
 'parval',
 'owner',
 'owner2',
 'mailadd',
 'mail_city',
 'mail_state',
 'mail_zip',
 'address',
 'saddno',
 'saddpref',
 'saddstr',
 'saddsttyp',
 'saddstsuf',
 'sunit',
 'scity',
 'original_a',
 'city',
 'county',
 'state2',
 'szip',
 'address_so',
 'legaldesc',
 'path',
 'll_uuid',
 'll_updated',
 'homeowner_',
 'lot_area',
 'pers_prop_',
 'tax_area_c',
 'contract_n',
 'contract_y',
 'non_renewa',
 'use_high_b',
 'use_second',
 'qoz',
 'qoz_tract',
 'll_gisacre',
 'OBJECTID_1',
 'DWR_revise',
 'Symb_class',
 'MULTIUSE',
 'CLASS1',
 'SUBCLASS1',
 'SPECOND1',
 'IRR_TYP1PA',
 'IRR_TYP1PB',
 'PCNT1',
 'CLASS2',
 'SUBCLASS2',
 'SPECOND2',
 'IRR_TYP2PA',
 'IRR_TYP2PB',
 'PCNT2',
 'CLASS3',
 'SUBCLASS3',
 'SPECOND3',
 'IRR_TYP3PA',
 'IRR_TYP3PB',
 'PCNT3',
 'UCF_ATT',
 'CROPTYP1',
 'CROPTYP2',
 'CROPTYP3',
 'Region',
 'Acres',
 'County_1',
 'Source',
 'Crop2016',
 'Modified_B',
 'Date_Data_',
 'Last_Modif',
 'GlobalID',
 'Shap

In [98]:
#get croplist
set(ld['Crop2016'].tolist())

{'Alfalfa and Alfalfa Mixtures',
 'Almonds',
 'Apples',
 'Avocados',
 'Beans (Dry)',
 'Bush Berries',
 'Carrots',
 'Cherries',
 'Citrus',
 'Cole Crops',
 'Corn, Sorghum and Sudan',
 'Cotton',
 'Dates',
 'Flowers, Nursery and Christmas Tree Farms',
 'Grapes',
 'Greenhouse',
 'Idle',
 'Kiwis',
 'Lettuce/Leafy Greens',
 'Managed Wetland',
 'Melons, Squash and Cucumbers',
 'Miscellaneous Deciduous',
 'Miscellaneous Field Crops',
 'Miscellaneous Grain and Hay',
 'Miscellaneous Grasses',
 'Miscellaneous Subtropical Fruits',
 'Miscellaneous Truck Crops',
 'Mixed Pasture',
 'Olives',
 'Onions and Garlic',
 'Peaches/Nectarines',
 'Pears',
 'Peppers',
 'Pistachios',
 'Plums, Prunes and Apricots',
 'Pomegranates',
 'Potatoes and Sweet Potatoes',
 'Rice',
 'Safflower',
 'Strawberries',
 'Sunflowers',
 'Tomatoes',
 'Urban',
 'Walnuts',
 'Wheat',
 'Young Perennials'}

So we are going to take the crop type -> get net water use per crop/1000 acres -> get  yield per acre per crop -> use to calculate parcel specific values for net water use and yeild -> then use price per unit or value of production/acre to get to a water value?

In [99]:
#first load in crop yield data
crop_yield = pd.read_csv('data/cacropyielddata.csv')

In [100]:
crop_yield.columns

Index(['Commodity', 'Planted All Purpose Acres', 'Harvested Acres', 'Yield',
       'Production', 'Price per Unit', 'Value of Production in Dollars'],
      dtype='object')

All of the crops are upper case so to make it easy we will upcase all of our crops in the dataframe

In [101]:
ld['Crop2016'] = ld['Crop2016'].str.upper()
our_crops = set(ld['Crop2016'].tolist())
our_crops

{'ALFALFA AND ALFALFA MIXTURES',
 'ALMONDS',
 'APPLES',
 'AVOCADOS',
 'BEANS (DRY)',
 'BUSH BERRIES',
 'CARROTS',
 'CHERRIES',
 'CITRUS',
 'COLE CROPS',
 'CORN, SORGHUM AND SUDAN',
 'COTTON',
 'DATES',
 'FLOWERS, NURSERY AND CHRISTMAS TREE FARMS',
 'GRAPES',
 'GREENHOUSE',
 'IDLE',
 'KIWIS',
 'LETTUCE/LEAFY GREENS',
 'MANAGED WETLAND',
 'MELONS, SQUASH AND CUCUMBERS',
 'MISCELLANEOUS DECIDUOUS',
 'MISCELLANEOUS FIELD CROPS',
 'MISCELLANEOUS GRAIN AND HAY',
 'MISCELLANEOUS GRASSES',
 'MISCELLANEOUS SUBTROPICAL FRUITS',
 'MISCELLANEOUS TRUCK CROPS',
 'MIXED PASTURE',
 'OLIVES',
 'ONIONS AND GARLIC',
 'PEACHES/NECTARINES',
 'PEARS',
 'PEPPERS',
 'PISTACHIOS',
 'PLUMS, PRUNES AND APRICOTS',
 'POMEGRANATES',
 'POTATOES AND SWEET POTATOES',
 'RICE',
 'SAFFLOWER',
 'STRAWBERRIES',
 'SUNFLOWERS',
 'TOMATOES',
 'URBAN',
 'WALNUTS',
 'WHEAT',
 'YOUNG PERENNIALS'}

Now let's compare the crops in our crop yield table with our the crops in our dataset

In [102]:
all_crops = set(crop_yield['Commodity'].tolist())
all_crops

{'ALFALFA',
 'APPLES',
 'APRICOTS',
 'ARTICHOKES',
 'ARTICHOKES, FRESH MARKET',
 'ARTICHOKES, UTILIZED',
 'ASPARAGUS',
 'ASPARAGUS, FRESH MARKET',
 'ASPARAGUS, UTILIZED',
 'BARLEY',
 'BEANS, SNAP',
 'BEANS, SNAP, FRESH MARKET',
 'BEANS, SNAP, PROCESSING',
 'BEANS, SNAP, UTILIZED',
 'BROCCOLI',
 'BROCCOLI, FRESH MARKET',
 'BROCCOLI, PROCESSING',
 'BROCCOLI, UTILIZED',
 'CABBAGE',
 'CABBAGE, FRESH MARKET',
 'CABBAGE, PROCESSING',
 'CABBAGE, UTILIZED',
 'CARROTS',
 'CARROTS, FRESH MARKET',
 'CARROTS, PROCESSING',
 'CARROTS, UTILIZED',
 'CAULIFLOWER',
 'CAULIFLOWER, FRESH MARKET',
 'CAULIFLOWER, PROCESSING',
 'CAULIFLOWER, UTILIZED',
 'CELERY',
 'CELERY, FRESH MARKET',
 'CELERY, PROCESSING',
 'CELERY, UTILIZED',
 'CHERRIES, SWEET',
 'CHICKPEAS',
 'CORN',
 'CORN, GRAIN',
 'CORN, SILAGE',
 'COTTON',
 'COTTON, COTTONSEED',
 'COTTON, PIMA',
 'COTTON, UPLAND',
 'CUCUMBERS',
 'CUCUMBERS, FRESH MARKET',
 'CUCUMBERS, PROCESSING',
 'CUCUMBERS, UTILIZED',
 'GARLIC',
 'GARLIC, FRESH MARKET',
 'GARLIC

In [103]:
overlap = our_crops.intersection(all_crops)
overlap

{'APPLES',
 'CARROTS',
 'COTTON',
 'GRAPES',
 'PEARS',
 'RICE',
 'SAFFLOWER',
 'TOMATOES',
 'WHEAT'}

Seems like quite a few are missing, lets see which ones...

In [104]:
our_remaining = our_crops - overlap

all_remaining = all_crops - overlap

print('Crops not matched in our dataset:\n', ', '.join(sorted(list(our_remaining))), '\n')

print('Crops left in the CA dataset:\n', ', '.join(sorted(list(all_remaining))))


Crops not matched in our dataset:
 ALFALFA AND ALFALFA MIXTURES, ALMONDS, AVOCADOS, BEANS (DRY), BUSH BERRIES, CHERRIES, CITRUS, COLE CROPS, CORN, SORGHUM AND SUDAN, DATES, FLOWERS, NURSERY AND CHRISTMAS TREE FARMS, GREENHOUSE, IDLE, KIWIS, LETTUCE/LEAFY GREENS, MANAGED WETLAND, MELONS, SQUASH AND CUCUMBERS, MISCELLANEOUS DECIDUOUS, MISCELLANEOUS FIELD CROPS, MISCELLANEOUS GRAIN AND HAY, MISCELLANEOUS GRASSES, MISCELLANEOUS SUBTROPICAL FRUITS, MISCELLANEOUS TRUCK CROPS, MIXED PASTURE, OLIVES, ONIONS AND GARLIC, PEACHES/NECTARINES, PEPPERS, PISTACHIOS, PLUMS, PRUNES AND APRICOTS, POMEGRANATES, POTATOES AND SWEET POTATOES, STRAWBERRIES, SUNFLOWERS, URBAN, WALNUTS, YOUNG PERENNIALS 

Crops left in the CA dataset:
 ALFALFA, APRICOTS, ARTICHOKES, ARTICHOKES, FRESH MARKET, ARTICHOKES, UTILIZED, ASPARAGUS, ASPARAGUS, FRESH MARKET, ASPARAGUS, UTILIZED, BARLEY, BEANS, SNAP, BEANS, SNAP, FRESH MARKET, BEANS, SNAP, PROCESSING, BEANS, SNAP, UTILIZED, BROCCOLI, BROCCOLI, FRESH MARKET, BROCCOLI, PRO

We need to fix some naming issues here for alfalfa, beans, corn, lettuce, 'melons, squash, and cucumber', miscellaneous grain and hay, garlic, peaches, prunes and apricots, potatoes and sweet potatoes, sunflower, we also have to select the crop data which has a value of production in dollars (revenue) value so we can get the value in dollars per acre foot of water applied to the crop. 

In [105]:
ld = ld.replace({'ALFALFA AND ALFALFA MIXTURES':'ALFALFA', 'BEANS (DRY)': 'BEANS, SNAP', 
            'CORN, SORGHUM AND SUDAN':'CORN, GRAIN', 'LETTUCE/LEAFY GREENS':'LETTUCE, HEAD', 
            'MISCELLANEOUS GRAIN AND HAY': 'HAY, (EXCL ALFALFA)', 'ONIONS AND GARLIC': 'GARLIC', 
            'PEACHES/NECTARINES': 'PEACHES', 'PLUMS, PRUNES AND APRICOTS': 'PRUNES', 
            'POTATOES AND SWEET POTATOES': 'POTATOES', 'SUNFLOWERS': 'SUNFLOWER', 'TOMATOES': 'TOMATOES, IN THE OPEN', 
            'COTTON': 'COTTON, COTTONSEED'})



In [106]:
set(ld['Crop2016'].tolist())

{'ALFALFA',
 'ALMONDS',
 'APPLES',
 'AVOCADOS',
 'BEANS, SNAP',
 'BUSH BERRIES',
 'CARROTS',
 'CHERRIES',
 'CITRUS',
 'COLE CROPS',
 'CORN, GRAIN',
 'COTTON, COTTONSEED',
 'DATES',
 'FLOWERS, NURSERY AND CHRISTMAS TREE FARMS',
 'GARLIC',
 'GRAPES',
 'GREENHOUSE',
 'HAY, (EXCL ALFALFA)',
 'IDLE',
 'KIWIS',
 'LETTUCE, HEAD',
 'MANAGED WETLAND',
 'MELONS, SQUASH AND CUCUMBERS',
 'MISCELLANEOUS DECIDUOUS',
 'MISCELLANEOUS FIELD CROPS',
 'MISCELLANEOUS GRASSES',
 'MISCELLANEOUS SUBTROPICAL FRUITS',
 'MISCELLANEOUS TRUCK CROPS',
 'MIXED PASTURE',
 'OLIVES',
 'PEACHES',
 'PEARS',
 'PEPPERS',
 'PISTACHIOS',
 'POMEGRANATES',
 'POTATOES',
 'PRUNES',
 'RICE',
 'SAFFLOWER',
 'STRAWBERRIES',
 'SUNFLOWER',
 'TOMATOES, IN THE OPEN',
 'URBAN',
 'WALNUTS',
 'WHEAT',
 'YOUNG PERENNIALS'}

Now we should look at the overlap again and drop the rows that dont overlap

In [107]:
our_crops = set(ld['Crop2016'].tolist())

overlap = our_crops.intersection(all_crops)
overlap

{'ALFALFA',
 'APPLES',
 'BEANS, SNAP',
 'CARROTS',
 'CORN, GRAIN',
 'COTTON, COTTONSEED',
 'GARLIC',
 'GRAPES',
 'HAY, (EXCL ALFALFA)',
 'LETTUCE, HEAD',
 'PEACHES',
 'PEARS',
 'POTATOES',
 'PRUNES',
 'RICE',
 'SAFFLOWER',
 'SUNFLOWER',
 'TOMATOES, IN THE OPEN',
 'WHEAT'}

In [118]:
incrop = list(overlap)

available_crops = ld['Crop2016'].isin(incrop)

matched = ld[available_crops]

In [119]:
len(matched)

14716

So with the current availble crop data we were able to match 14,967 records. Now lets also prune the CA data and merge it into our land data 

In [120]:
available_commodity = crop_yield['Commodity'].isin(incrop)

CA_prune = crop_yield[available_commodity]



In [121]:
#rename columns to merge

CA_prune.rename(columns = {'Commodity': 'Crop'}, inplace=True)

matched.rename(columns = {'Crop2016': 'Crop'}, inplace=True)



/Users/d/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [227]:
#now merge

merged = matched.merge(CA_prune, on='Crop')

In [228]:
merged.columns

Index(['geoid', 'parcelnumb', 'usecode', 'improvval', 'landval', 'parval',
       'owner', 'owner2', 'mailadd', 'mail_city', 'mail_state', 'mail_zip',
       'address', 'saddno', 'saddpref', 'saddstr', 'saddsttyp', 'saddstsuf',
       'sunit', 'scity', 'original_a', 'city', 'county', 'state2', 'szip',
       'address_so', 'legaldesc', 'path', 'll_uuid', 'll_updated',
       'homeowner_', 'lot_area', 'pers_prop_', 'tax_area_c', 'contract_n',
       'contract_y', 'non_renewa', 'use_high_b', 'use_second', 'qoz',
       'qoz_tract', 'll_gisacre', 'OBJECTID_1', 'DWR_revise', 'Symb_class',
       'MULTIUSE', 'CLASS1', 'SUBCLASS1', 'SPECOND1', 'IRR_TYP1PA',
       'IRR_TYP1PB', 'PCNT1', 'CLASS2', 'SUBCLASS2', 'SPECOND2', 'IRR_TYP2PA',
       'IRR_TYP2PB', 'PCNT2', 'CLASS3', 'SUBCLASS3', 'SPECOND3', 'IRR_TYP3PA',
       'IRR_TYP3PB', 'PCNT3', 'UCF_ATT', 'CROPTYP1', 'CROPTYP2', 'CROPTYP3',
       'Region', 'Acres', 'County_1', 'Source', 'Crop', 'Modified_B',
       'Date_Data_', 'Last_Modif', '

Now we just need to create the gross water usage column using average acre feet of water applied per acreage per crop.

In [229]:
 applied_water = {'ALFALFA': 5.2,
 'APPLES': 2.2,
 'BEANS, SNAP': 0.2,
 'CARROTS': 1.3,
 'CORN, GRAIN': 2.2,
 'COTTON, COTTONSEED': 0.9,
 'GARLIC': 0.2,
 'GRAPES': 1.6,
 'HAY, (EXCL ALFALFA)': 0.8,
 'LETTUCE, HEAD': 1.3,
 'PEACHES': 2.2,
 'PEARS': 2.2,
 'POTATOES': 0.1,
 'PRUNES': 2.2,
 'RICE': 2.7,
 'SAFFLOWER': 0.1,
 'SUNFLOWER': 1.7,
 'TOMATOES, IN THE OPEN': 0.1,
 'WHEAT': 0.8}

source used for this value
https://pacinst.org/wp-content/uploads/2015/04/CA-Ag-Water-Use.pdf



Now we add two columns to our dataset to hold our applied water in millions of acre feet and our water productivity column and fill in both

In [230]:
merged['water_productivity'] = None
merged['applied_water_mil'] = None

In [231]:
#add in applied water in millions acre feet
for i, r in merged.iterrows():
    
    applied = applied_water[r['Crop']]
    merged.at[i, 'applied_water_mil'] = applied

In [232]:
#clean up Value of Production in Dollars

print('Before:', set(merged['Value of Production in Dollars'].tolist()))

merged.dropna(subset=['Value of Production in Dollars'], inplace = True)



Before: {nan, '23,897,000', '253,227,000', '19,094,000', '1,174,395,000', '47,766,000', '844,190,000', '308,777,000', '98,820,000', '44,336,000', '874,290,000', '327,277,000', '827,126,000', '45,864,000', '775,761,000'}


In [238]:
for i, r in merged.iterrows():
    
    stripped = r['Value of Production in Dollars'].replace(',', '')
    merged.at[i, 'Value of Production in Dollars'] = int(stripped)

In [239]:
print('After:', set(merged['Value of Production in Dollars'].tolist()))

After: {44336000, 98820000, 45864000, 327277000, 308777000, 775761000, 23897000, 844190000, 827126000, 874290000, 47766000, 253227000, 19094000, 1174395000}


In [242]:
#now divide millions by millions to get a revenue per acre feet of water applied
for i, r in merged.iterrows():

#try:
        
    converted = r['applied_water_mil'] * 1000000

    merged.at[i, 'water_productivity'] = float(r['Value of Production in Dollars'])/converted

#except:
  

        

In [243]:
merged['water_productivity']

7325     162.344
7326     162.344
7327     162.344
7328     162.344
7329     162.344
          ...   
14711     238.97
14712     238.97
14713     238.97
14714     238.97
14715     238.97
Name: water_productivity, Length: 5567, dtype: object